In [16]:
from sklearn.cluster import KMeans
from collections import OrderedDict
from sklearn import datasets, metrics
import numpy as np
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt
import Handout_MNIST.MNIST as mn


In [17]:
# in this cell you will have to add a suitable intialisation for the EM-algorithm to find good "clusters"
# note that "cluster" is a bit misleading, as only the degree of belonging to a distribution is evaluated 
# for each sample, and not an absolute assignment to a cluster is made

class EMClusters:

    def __init__(self):
        self.__numOfClasses = 0
        self.__numOfAttributes = 0

    # Initialises for EM to work
    # should be called with a suitable smoothing factor blur, but the default does at least something.
    def initialise(self, data, num_classes, num_attributes, blur=0.1):
        self.__blur = blur
        self.__data = data
        self.__numOfAttributes = num_attributes
  
        self.__numOfClasses = num_classes

        self.__clusterMeansAndCovs = np.ones((self.__numOfClasses, self.__numOfAttributes, 2), dtype=float)
        self.__priors = np.ones(self.__numOfClasses)

        # start of your code
        ###
        #
        # Here, you should now fill in something more sensible, as the code above is only a placeholder solution...
        # You need to find initial clusters and set start values for the priors, means and (co)variances that would
        # end up in the following arrays:
        #
        # self.__priors              - is of dimension K x 1 and holds K values, K the number of clusters / classes
        # self.__clusterMeansAndCovs - is of dimensions K x I x 2 and holds for each combination of class k
        #                              and attribute j the mean in [k, j, 0] and the covariance (only for jj,
        #                              as we assume conditional independence given the class) in [k, j, 1]
        #
        ###
        self.__priors = np.random.rand(self.__numOfClasses)
        self.__clusterMeansAndCovs = np.random.rand(self.__numOfClasses, self.__numOfAttributes, 2)
        # end of your code

        
    # the actual EM algorithm, should be called with a suitable eps for a stop criterion
    # feel free to change (optimize) the implementation, but you do not have to do that
    def fit(self, eps=1.0):
        epsilon = eps

        print(self.__numOfAttributes)

        # print(self.__clusterMeansAndCovs[:,1,:])

        resp = np.zeros((len(self.__data), self.__numOfClasses))
        r_k = np.zeros(self.__numOfClasses)
        notDone = True
        while notDone:
            # print(self.__clusterMeansAndCovs)
            # E-step
            for i in range(len(self.__data)):
                probs = self.__priors.copy()
                for k in range(self.__numOfClasses):

                    for attr in range(self.__numOfAttributes):
                        probs[k] *= 1.0 / np.sqrt(
                            2 * math.pi * (self.__clusterMeansAndCovs[k, attr, 1] + self.__blur)) * math.exp(
                            -1.0 * math.pow((self.__clusterMeansAndCovs[k, attr, 0] - self.__data[i, attr]), 2) / (
                                        self.__clusterMeansAndCovs[k, attr, 1] + self.__blur))

                resp[i, :] = probs / sum(probs)
                # print(resp[i,:])
            
            # M-step
            newClusterMeansAndCovs = np.zeros((self.__numOfClasses, self.__numOfAttributes, 2))
            for k in range(self.__numOfClasses):
                r_k[k] = sum(resp[:, k])
                self.__priors[k] = 1 / len(self.__data) * r_k[k]

                for j in range(self.__numOfAttributes):
                    means = sum(resp[:, k] * self.__data[:, j]) / r_k[k]

                    newClusterMeansAndCovs[k, j, 0] = means
                    newClusterMeansAndCovs[k, j, 1] = sum(resp[:, k] * self.__data[:, j] * self.__data[:, j]) \
                                                      / r_k[k] - newClusterMeansAndCovs[k, j, 0] ** 2

            # print(newClusterMeansAndCovs[:, :, 1])

            err = np.linalg.norm(self.__clusterMeansAndCovs[:, :, 0] - newClusterMeansAndCovs[:, :, 0])

            self.__clusterMeansAndCovs = newClusterMeansAndCovs.copy()
            print("err = " + str(err))
            if err <= epsilon:
                notDone = False
                
                
        # produce "clusters", i.e, assign the samples to "their" gaussian
        clustered = 10 * np.ones(len(self.__data), dtype=int)
        for i in range(len(self.__data)):
            probs = self.__priors.copy()
            for k in range(self.__numOfClasses):

                for attr in range(self.__numOfAttributes):
                    probs[k] *= 1.0 / np.sqrt(2 * math.pi * (self.__clusterMeansAndCovs[k, attr, 1] + self.__blur)) \
                                * math.exp(
                        -1.0 * math.pow((self.__clusterMeansAndCovs[k, attr, 0] - self.__data[i, attr]), 2) \
                        / (self.__clusterMeansAndCovs[k, attr, 1] + self.__blur))

            sumProb = np.sum(probs)
            if (sumProb > 0.0):
                probs = probs / sumProb
                clustered[i] = np.argmax(probs)

        return clustered, self.__clusterMeansAndCovs

    # prediction uses the found gaussians to compute the likelihoods in a GNBC
    def predict(self, samples):
        predicted = [None for i in range(len(samples))]
        prob = [[self.__priors[cls] for cls in range(self.__numOfClasses)] for i in range(len(samples))]
        # print(prob)
        for i in range(len(samples)):
            for cls in range(self.__numOfClasses):
                for attr in range(self.__numOfAttributes):
                    (mean, var) = (
                    self.__clusterMeansAndCovs[cls, attr, 0], self.__clusterMeansAndCovs[cls, attr, 1] + self.__blur)
                    # print( mean, var)
                    prob[i][cls] *= 1.0 / np.sqrt(2 * math.pi * var) * math.exp(
                        -1.0 * math.pow((mean - samples[i, attr]), 2) / var)
                # print(i, cls, prob[i][cls])

            sumProb = np.sum(prob[i])
            if (sumProb > 0.0):
                prob[i] = prob[i] / sumProb
                predicted[i] = np.argmax(prob[i])
            else:
                predicted[i] = 10

            # print(prob[i])

        return predicted

In [18]:
# use this to rearrange the outcome of a clustering to match the order of the classes in the training data
# note that this is only something to make it easier for you to inspect the results, but probably not a method
# you want to integrate in a final system

def correctClusters( confM, classes) :
    clusterMapping = -1 * np.ones_like(classes)
    for k in classes:
        temp = list(confM[:, k])
        #print(temp)
        notDone = True
        while notDone:
            amax = np.argmax(temp)
            if (confM[amax, k] == np.max(confM[amax, :])):
                clusterMapping[k] = amax
                notDone = False
            elif (sum(temp) == 0):
                notDone = False
            else:
                temp[amax] = 0

    for k in classes:
        if (clusterMapping[k] == -1):
            for j in classes:
                if j not in clusterMapping:
                    clusterMapping[k] = j

    return clusterMapping


In [19]:
# loading the digits data and normalising to values between 0 and 1

digits = datasets.load_digits()


num_examples = len(digits.data)
num_split = int(0.7 * num_examples)
train_features = 1/16. * digits.data[:num_split]
train_labels = digits.target[:num_split]
test_features = 1/16. *digits.data[num_split:]
test_labels = digits.target[num_split:]

num_attributes = 64

num_classes = 10

# alternatively, loading the MNIST_Light data, which is normalised in the get_data() method already
# OBS: this will take a while to run!

#mnist = mn.MNISTData('Handout_MNIST/MNIST_Light/*/*.png')
#train_features, test_features, train_labels, test_labels = mnist.get_data()
#num_attributes = 400
#num_classes = 10

The cells below contain a breakdown for what was meant to be executed in a single cell in the handout skeleton. The single cell contained the following description:

"In this cell you should add the k-Means clustering to be able to compare to what you got with EM
If you apply "correctClusters" from above (works with any confusion matrix), you can even test the results 
against your or SKLearn's results in a classification attempt."

Define a utility function for making use of the provided `correctClusters` function output.

In [5]:
def map_correct_labels(predicted_labels, correct_clusters):
    kmeans_predicted_corrected = []
    for pred in predicted_labels:
        kmeans_predicted_corrected.append(correct_clusters[pred])
    
    return kmeans_predicted_corrected

Define the set of classes.

In [6]:
classes = [i for i in range(num_classes)]

Do K-Means clustering.

In [7]:
clustering = KMeans(n_clusters=num_classes)
clustering.fit(train_features, train_labels)
kmeans_predicted = clustering.predict(train_features)
kmeans_confusion_matrix = metrics.confusion_matrix(train_labels, kmeans_predicted)
print('CONFUSION MATRIX BEFORE CORRECTION\n', kmeans_confusion_matrix)

CONFUSION MATRIX BEFORE CORRECTION
 [[  1   0   0   0 124   0   0   0   0   0]
 [  0   0   1   0   0   0  26   0  63  39]
 [  0   3   0   0   0   4 105   1  10   1]
 [  0 110   0   0   0   4   0  16   0   0]
 [108   0   0   0   0   7   0   0   2   7]
 [  1   2   0  95   0   0   0  28   0   0]
 [  0   0 124   0   1   0   0   0   2   0]
 [  0   0   0   1   0 122   0   0   0   2]
 [  0   1   1   3   0   2   2  42  67   4]
 [  0   3   0   2   0   7   0  97   1  15]]


Print the metrics of K-Means.

In [8]:
kmeans_correct_clusters = correctClusters(metrics.confusion_matrix(train_labels, kmeans_predicted), classes)
kmeans_predicted_corrected = map_correct_labels(kmeans_predicted, kmeans_correct_clusters)

kmeans_confusion_matrix_corrected = metrics.confusion_matrix(train_labels, kmeans_predicted_corrected)

print('K-MEANS CLASSIFICATION REPORT\n', metrics.classification_report(train_labels, kmeans_predicted_corrected))
print('CONFUSION MATRIX CORRECT KMEANS\n', kmeans_confusion_matrix_corrected)

K-MEANS CLASSIFICATION REPORT
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       125
           1       0.57      0.30      0.40       129
           2       0.79      0.85      0.82       124
           3       0.92      0.85      0.88       130
           4       0.98      0.87      0.92       124
           5       0.94      0.75      0.84       126
           6       0.98      0.98      0.98       127
           7       0.84      0.98      0.90       125
           8       0.46      0.55      0.50       122
           9       0.53      0.78      0.63       125

    accuracy                           0.79      1257
   macro avg       0.80      0.79      0.79      1257
weighted avg       0.80      0.79      0.79      1257

CONFUSION MATRIX CORRECT KMEANS
 [[124   0   0   0   1   0   0   0   0   0]
 [  0  39  26   0   0   0   1   0  63   0]
 [  0   1 105   3   0   0   0   4  10   1]
 [  0   0   0 110   0   0   0   4   0  16]
 [  0

In [9]:
# # print('EM CLASSIFICATION REPORT\n', metrics.classification_report(test_labels, em_predicted))
# em_confusion_matrix = metrics.confusion_matrix(train_labels, clustered)
# em_correct_clusters = correctClusters(metrics.confusion_matrix(train_labels, clustered), classes)
# clusters_corrected = map_correct_labels(clustered, em_correct_clusters)
# kmeans_confusion_matrix_corrected = metrics.confusion_matrix(train_labels, clusters_corrected)
# print('K-MEANS CLASSIFICATION REPORT\n', metrics.classification_report(train_labels, clusters_corrected))
# print('CONFUSION MATRIX CORRECT EM\n', kmeans_confusion_matrix_corrected)

In [20]:
# In this cell you should add the k-Means clustering to be able to compare to what you got with EM
# If you apply "correctClusters" from above (works with any confusion matrix), you can even test the results 
# against your or SKLearn's results in a classification attempt.

# clustering = KMeans(n_clusters=num_classes)
# clustering.fit(train_features, train_labels)
# kmeans_predicted = clustering.predict(test_features)
# # print('K-MEANS CLASSIFICATION REPORT\n', metrics.classification_report(test_labels, kmeans_predicted))
# kmeans_confusion_matrix = metrics.confusion_matrix(test_labels, kmeans_predicted)
# kmeans_correct_clusters = correctClusters(metrics.confusion_matrix(test_labels, kmeans_predicted), classes)
# kmeans_confusion_matrix_correct = swap_rows(kmeans_confusion_matrix, kmeans_correct_clusters)
# print('CONFUSION MATRIX CORRECT KMEANS\n', kmeans_confusion_matrix_correct)

emClusters = EMClusters()
emClusters.initialise(train_features, num_classes, num_attributes, blur=0.2)
clustered, clusterMeansVars = emClusters.fit(0.001)
# print('EM CLASSIFICATION REPORT\n', metrics.classification_report(test_labels, em_predicted))
# em_confusion_matrix = metrics.confusion_matrix(train_labels, clustered)

em_correct_clusters = correctClusters(metrics.confusion_matrix(train_labels, clustered), classes)
em_corrected_labels = map_correct_labels(clustered, em_correct_clusters)
em_confusion_matrix_corrected = metrics.confusion_matrix(train_labels, em_corrected_labels)
# em_confusion_matrix_correct = swap_rows(em_confusion_matrix, em_correct_clusters)
print('EM CLASSIFICATION REPORT\n', metrics.classification_report(train_labels, em_corrected_labels))
print('CONFUSION MATRIX EM\n', em_confusion_matrix_corrected)

64
err = 10.530723801775258
err = 2.1621474103712184
err = 1.048147812239279
err = 0.47758685062143535
err = 0.3735057816896035
err = 0.37702827146766277
err = 0.3530287337456704
err = 0.17291615265360233
err = 0.11509964754709107
err = 0.10172622739325972
err = 0.07415001548660599
err = 0.04970127326803362
err = 0.04067024983574612
err = 0.033714175032333336
err = 0.027904423969868544
err = 0.02366444918683144
err = 0.02054105078092677
err = 0.017938744182219718
err = 0.015496307545109734
err = 0.0131457851286789
err = 0.010986182165180297
err = 0.009122695938605992
err = 0.0075971880287400385
err = 0.006394159692739271
err = 0.005467107706054289
err = 0.004759822353639651
err = 0.004218901200914628
err = 0.003799599071648577
err = 0.003467402605849315
err = 0.0031971995097484075
err = 0.002971426085367676
err = 0.002778078579546695
err = 0.002609021990645908
err = 0.002458718847920556
err = 0.002323342302114382
err = 0.0022001851831497153
err = 0.002087276777349579
err = 0.0019831375

In [23]:
completeness_score = metrics.completeness_score(train_labels, clustered)
homogeneity_score = metrics.homogeneity_score(train_labels, clustered)
mutual_info_score = metrics.adjusted_mutual_info_score(train_labels, clustered)
print( "Completeness, homogeneity, adj mutual info EM vs labels", completeness_score, homogeneity_score, mutual_info_score)


Completeness, homogeneity, adj mutual info EM vs labels 0.7519930900662148 0.7119746383067759 0.7274200859434542


In [24]:
em_confusion_matrix_corrected = metrics.confusion_matrix(kmeans_predicted_corrected, em_corrected_labels)
# em_confusion_matrix_correct = swap_rows(em_confusion_matrix, em_correct_clusters)
print('EM ON KMEANS CLASSIFICATION REPORT\n', metrics.classification_report(kmeans_predicted_corrected, em_corrected_labels))
print('CONFUSION MATRIX EM ON KMEANS\n', em_confusion_matrix_corrected)

EM CLASSIFICATION REPORT
               precision    recall  f1-score   support

           0       1.00      0.70      0.82       125
           1       1.00      0.94      0.97        68
           2       0.99      0.98      0.99       133
           3       0.40      0.97      0.56       119
           4       0.97      0.99      0.98       110
           5       0.98      0.98      0.98       101
           6       0.99      0.99      0.99       126
           7       0.97      0.97      0.97       146
           8       0.93      1.00      0.96       145
           9       0.00      0.00      0.00       184

    accuracy                           0.81      1257
   macro avg       0.82      0.85      0.82      1257
weighted avg       0.78      0.81      0.78      1257

CONFUSION MATRIX CORRECT EM
 [[ 87   0   0   0   0   0   0   0   0  38]
 [  0  64   0   0   3   0   0   1   0   0]
 [  0   0 131   0   0   0   1   0   1   0]
 [  0   0   0 116   0   0   0   2   1   0]
 [  0   0   0 